# L-Type Calcium Channel

Get document information for documents comprising LTCC data set.

In [10]:
from sqlalchemy import create_engine

import requests

import lxml.html

import re

In [11]:
# ChEMBL connection...

engine = create_engine(open('database.txt').read().strip())

Get document info for symbol...

In [12]:
docs = pd.read_sql("""
select
    a.doc_chemblid
  , b.pubmed_id
  , c.title
  , c.abstract
  , b.journal || ', v. ' || b.volume || ', p. ' || b.first_page || ' (' || b.year || ')' as citation
  , b.doi
from
  (
    select distinct
      doc_chemblid
    from
      TT_LTCC_CURVE_DATA
  ) a
  join chembl_20_app.docs b on a.doc_chemblid = b.chembl_id
  left outer join chembl_20_app.journal_articles c on b.pubmed_id = c.pubmed_id
  left outer join chembl_20_app.journals d on c.journal_id = d.journal_id
order by
    a.doc_chemblid
""", engine).set_index('doc_chemblid', drop=True)

Add link to DOI where, available...

In [14]:
docs['doi'] = docs['doi'].apply(lambda x: '<a target="_blank" href="http://dx.doi.org/{}">{}</a>'.format(*(x,)*2) if x is not None else '')

Add direct link(s) to publisher's web page for article, where available...

See notebook '[Links](links.ipynb)' for details.

In [22]:
# Save intermediate docs table for notebook 'Links.ipynb'...

docs.to_pickle('docs_0.pkl')

Run notebook '[Links](links.ipynb)'

In [23]:
# Load info generated in notebook 'Links.ipynb'...

links = pd.read_pickle('links.pkl')

docs = docs.merge(links, how='left', left_on='pubmed_id', right_index=True)

In [24]:
for col in ['title', 'abstract', 'links']: docs[col] = docs[col].fillna('')

In [25]:
docs['citation'] = docs['citation'].apply(lambda x: x.replace(' ', '&nbsp;'))

In [26]:
HTML(docs.to_html())

,pubmed_id,title,abstract,citation,doi,links
doc_chemblid,,,,,,
CHEMBL1123385,3023614,"Stereoselectivity of a potent calcium antagonist, 1-benzyl-3-pyrrolidinyl methyl 2,6-dimethyl-4-(m-nitrophenyl)-1,4-dihydropyridine-3,5-dicarboxylate.","Four enantiomers (3a-d) of the title compound, YM-09730 (3), were synthesized by the reaction of (-)- or (+)-5-(methoxycarbonyl)-2, 6-dimethyl-4-(m-nitrophenyl)-1,4-dihydropyridine-3-carboxylic acid (1a or 1b) with (S)- or (R)-1-benzyl-3-pyrrolidinol (2a or 2b). [3H]Nitrendipine binding affinity and coronary vasodilating activity of these compounds were evaluated. The absolute configuration of the most potent enantiomer (3a) with the longest duration was unequivocally determined to be (S)-1,4-dihydropyridine-C4 and (S)-pyrrolidine-C3 (S,S) by X-ray crystallographic study on 3a X HBr as well as 3a X HCl. The configuration of 1a corresponds to R, and the other enantiomers of 3 were respectively determined by chemical correlation. The potency order of the four enantiomers was (S,S)-3a greater than (S,R)-3b greater than (R,R)-3d greater than (R,S)-3c. Latent chiral characters of nifedipine derivatives with the identical ester groups were assigned by comparison of their puckering modes of 1,4-dihydropyridine (DHP) rings with those found in 3a X HCl or 3a X HBr. On the basis of the assignment, it has been revealed that the (S)-DHP nifedipine derivatives possess the synperiplanar carbonyl group at C5. The conformational restriction may be a factor causing stereoselectivity of antagonism.","J. Med. Chem., v. 29, p. 2504 (1986)",,"Link_1 <a target=""_blank"" href=""http://wtgcsfx.hosted.exlibrisgroup.com/wtsc/cgi/public/docdel.cgi?source=Entrez%3APubMed+(Via+SFX)&meduid=3023614&month=12&author=Tamazawa%2C+K&issue=12&ISBN=&article=Stereoselectivity+of+a+potent+calcium+antagonist%2C+1-benzyl-3-pyrrolidinyl+methyl+2%2C6-dimethyl-4-(m-nitrophenyl)-1%2C4-dihydropyridine-3%2C5-dicarboxylate.&publisher=American+Chemical+Society&journal=Journal+of+medicinal+chemistry&ISSN=0022-2623&abbrev=J+MED+CHEM&volume=29&recipient=libadmin%40sanger.ac.uk&genre=article&publiPlace=%5BEaston%2C+Pa.%5D&sender=&year=1986&pages=2504-11"">Link_2"
CHEMBL1123712,2435903,"Synthesis and biological activity of novel calcium channel blockers: 2,5-dihydro-4-methyl-2-phenyl-1,5-benzothiazepine-3-carboxylic acid esters and 2,5-dihydro-4-methyl-2-phenyl-1,5-benzodiazepine-3-carboxylic acid esters.","2,5-Dihydro-4-methyl-2-phenyl-1,5-benzothiazepine-3-carboxylic acid esters, based on the structures of dihydropyridines and diltiazem, were synthesized from o-aminothiophenol and 2-(phenylmethylene)- 3-oxobutanoic acid esters. Biological evaluation in the potassium-depolarized rabbit aorta suggests that these compounds are calcium channel blockers. The in vitro activity was further confirmed by electrophysiological techniques. Structure-activity studies for the aromatic substitution showed that the 2-nitro derivative was the most potent (IC50 = 0.3 microM) compound in vitro while the ethyl ester was slightly better than the corresponding methyl ester. Replacement of sulfur with nitrogen atom provided 2,5-dihydro-4-methyl-2-(3-nitrophenyl)-1,5-benzodiazepine-3-carboxylic acid ethyl ester, which was only slightly less active than the corresponding benzothiazepine. Derivatization of the nitrogen in 2,5-dihydro-4-methyl- 2-(3-nitrophenyl)-1,5-benzothiazepine-3-carboxylic acid methyl ester with a (dimethylamino)ethyl group (present in diltiazem) provided 2,5-dihydro-5-[(dimethylamino)ethyl]- 4-methyl-2-(3-nitrophenyl)-1,5-benzo-thiazepine-3-carboxylic acid methyl ester, which was found to be equipotent to diltiazem in vitro. Radioligand binding studies using [3H]nitrendipine and [3H]diltiazem showed that the compound with the free nitrogen binds competitively into the dihydropyridine binding site while the molecule in which the nitrogen is alkylated with a (dimethylamino)ethyl group interacts competitively with both diltiazem and dihydropyridine 

In [27]:
# Save fginakl version of docs table for main notebook...

docs.to_pickle('docs.pkl')